In [2]:
import os


In [3]:
%pwd


'k:\\pipeline\\research'

In [4]:
os.chdir("../")


In [5]:
%pwd


'k:\\pipeline'

In [ ]:
# entity is the return type of a function
#we can also create custom return types using classes
###### put this in config_entity.py file ######
from dataclasses import dataclass
from pathlib import Path

#so now its not a python actual class its a dataclass
#frozen means we cannot change the values of the attributes once we create an object    
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
#from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [14]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [ ]:
### put this in configuration.py file inside src/config ######

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #a folder with name artifact will be created


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [24]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [ ]:
#### put this in data_ingestion.py file inside src/components ######

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, #downloads from here
                filename = self.config.local_data_file #saves to this location
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
from pathlib import Path

config = read_yaml(Path("config/config.yaml"))  # ✅ Correct location


[2025-09-08 12:32:24,136: INFO: common: yaml file: config\config.yaml loaded successfully]


In [27]:
try:
    # Step 1: Load configuration
    config = ConfigurationManager()

    # Step 2: Get data ingestion specific config
    data_ingestion_config = config.get_data_ingestion_config()

    # Step 3: Initialize data ingestion pipeline
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # Step 4: Download and extract dataset
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    # Optional: log error or handle gracefully
    raise e


[2025-09-08 12:32:24,692: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-08 12:32:24,698: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-08 12:32:24,699: INFO: common: created directory at: artifacts]
[2025-09-08 12:32:24,699: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-08 12:32:24,700: INFO: 2109712141: File already exists of size: ~ 11361 KB]
